In [2]:
import random
import re
import os
import tqdm
import xml.etree.ElementTree as ET

import numpy as np
from tensorflow import keras
from nltk import tokenize
from tensorflow.keras.utils import Sequence

In [3]:
class Debug:
    def __init__(self, debug_mode=True):
        self.debug_mode = debug_mode
        self.flag = {}

    def log(self, target, flag=None):
        if self.debug_mode:
            if flag is None:
                print(target)
            else:
                if flag in self.flag.keys():
                    if self.flag[flag]:
                        print(target)

    def set_flag(self, flag: str, val: bool):
        self.flag[flag] = val

debug = Debug(True)

In [40]:
class GeneratorExceptions(Exception):
    """
    The Exception class for tracking all exceptions raised in data generator
    Param
        text: the displayed text
    """
    def __init__(self, text: str):
        self.text = text

class data_generator(Sequence):
    def __init__(self, 
                 dataset_file_path : str="data/dataset/nysk.xml", 
                 processed_dataset_path: str ="data/processed_dataset/",
                 batch_size = 1,
                 shuffle = True):
        self.dataset_file_path = dataset_file_path
        self.processed_dataset_path = processed_dataset_path
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.file_list = []
        self.encoder_input_data = []
        self.decoder_input_data = []
        self.decoder_target_data = []
        self.char_list = []
        self.characters_set = set()
        self.tokens_count = 0
        self.max_sequence_len = 0
        self.token_index = None
        self.input_texts = None
        
        self.validation_split=0.1
        
    def preprocess_data(self, override=False, max:int=-1):
        self.file_list = []
        if os.path.isfile(self.dataset_file_path):
            if not os.path.isdir(self.processed_dataset_path):
                os.mkdir(self.processed_dataset_path)

            with open(self.dataset_file_path, "r", encoding="utf-8") as f:
                doc = ET.ElementTree(file=f)

            root = doc.getroot()
            
            pos = 0
            
            for item in tqdm.tqdm(root):
                if max != -1:
                    if pos > max:
                        break
                    pos += 1
                news_id = item.findtext('docid')
                source = item.findtext('source')
                url = item.findtext('url')
                title = item.findtext('title')
                summary = item.findtext('summary')
                text = item.findtext('text')

                title = re.sub(r"<.*>", "", title)
                title = re.sub(r"\W", "_", title)
                title = f"{news_id}_{title[:10]}"
                
                res = tokenize.sent_tokenize(text)
                sentences_count = len(res)
                
                for s in res:
                    if len(s) > self.max_sequence_len:
                        self.max_sequence_len = len(s)
                
                self.max_sequence_len += 2
                
                for c in text:
                    if c not in self.characters_set:
                        self.characters_set.add(c)
                
                self.characters_set.add('\t')
                self.characters_set.add('\n')
                
                fp = f"{self.processed_dataset_path}{title}_{sentences_count}.txt"
                
                if not os.path.isfile(fp) or override:
                    with open(fp, 'w', encoding='utf-8') as f:
                        f.write(text)
            
            self.char_list = sorted(list(self.characters_set))
            self.tokens_count = len(self.char_list)
            self.token_index = dict([(char, i) for i, char in enumerate(self.characters_set)])
        else:
            raise GeneratorExceptions("Path doesn't exist")
    
    def generate_file_list(self, length:int=-1):
        temp = os.listdir(self.processed_dataset_path)
        self.file_list = []
        t_list = []
        for i in temp:
            t_list.append(f"{self.processed_dataset_path}{i}")
        self.file_list = t_list[1:length]
    
    def process_data(self, text):
        input_texts = []
        target_texts = []
        
        for i in range(0, len(text)-1):
            input_t = f"\t{text[i]}\n"
            target_t = f"\t{text[i+1]}\n"
            input_texts.append(input_t)
            target_texts.append(target_t)
        
        temp_encoder_input_data = np.zeros(
            (len(input_texts), self.max_sequence_len, self.tokens_count), dtype="float32"
        )
        temp_decoder_input_data = np.zeros(
            (len(input_texts), self.max_sequence_len, self.tokens_count), dtype="float32"
        )
        temp_decoder_target_data = np.zeros(
            (len(input_texts), self.max_sequence_len, self.tokens_count), dtype="float32"
        )

        for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
            for t, char in enumerate(input_text):
                temp_encoder_input_data[i, t, self.token_index[char]] = 1.0
                temp_encoder_input_data[i, t + 1 :, self.token_index[" "]] = 1.0
            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                temp_decoder_input_data[i, t, self.token_index[char]] = 1.0
                if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                    temp_decoder_target_data[i, t - 1, self.token_index[char]] = 1.0
                    temp_decoder_input_data[i, t + 1 :, self.token_index[" "]] = 1.0
                    temp_decoder_target_data[i, t:, self.token_index[" "]] = 1.0
        
        self.encoder_input_data = temp_encoder_input_data
        self.decoder_input_data = temp_decoder_input_data
        self.decoder_target_data = temp_decoder_target_data
        
        self.input_texts = input_texts
        
        return
    
    def __len__(self):
        return len(self.file_list) // self.batch_size
    
    def __getitem__(self, index):
        files = self.file_list[index * self.batch_size:(index + 1) * self.batch_size]
        res = []
        for fl in files:
            with open(fl, 'r', encoding='utf-8') as dt:
                text = dt.read()
                temp_list = tokenize.sent_tokenize(text)
                res.extend(temp_list)
        
        self.process_data(res)
        
        return [self.encoder_input_data, self.decoder_input_data], self.decoder_target_data,
    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.file_list)
        return
    

In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
# Testing data generator
DataGenerator = data_generator()
DataGenerator.preprocess_data(max=200)

  2%|▏         | 201/10421 [00:00<00:15, 671.42it/s]


In [43]:
print(DataGenerator.tokens_count)
print(DataGenerator.max_sequence_len)

137
2985


In [44]:
DataGenerator.generate_file_list(200)
print(len(DataGenerator))
print(len(DataGenerator.file_list))
# print(temp_dg.__getitem__(1))

199
199


In [45]:
def generate_model(num_encoder_tokens, num_decoder_tokens, latent_dim=256):
      
    encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
    encoder = keras.layers.LSTM(latent_dim, return_state=True)
    encoder_outputs_, state_h, state_c = encoder(encoder_inputs)
    
    encoder_states = [state_h, state_c]
    
    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
    
    decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    model.summary()
    
    return model

In [46]:
latent_dim = 256

model = generate_model(num_encoder_tokens=DataGenerator.tokens_count, 
                       num_decoder_tokens=DataGenerator.tokens_count,
                       latent_dim=latent_dim)

model_name = "Model\SeqToSeq_Model"

model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None, 137)]  0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None, 137)]  0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, 256), (None, 403456      input_9[0][0]                    
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, None, 256),  403456      input_10[0][0]                   
                                                                 lstm_8[0][1]          

In [47]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [48]:
# Don't execute this one for now
import nltk
nltk.download('punkt')

epochs = 1  

print(DataGenerator.token_index)

model.fit(DataGenerator, epochs=epochs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'Б': 0, 'K': 1, 'd': 2, 'J': 3, '©': 4, '^': 5, 'R': 6, 'à': 7, 'B': 8, 'é': 9, 'k': 10, '?': 11, '`': 12, '9': 13, '€': 14, '"': 15, 'н': 16, '‘': 17, '«': 18, '\n': 19, '5': 20, 'x': 21, 'w': 22, 'D': 23, '}': 24, '·': 25, 'T': 26, '+': 27, '…': 28, 'g': 29, 'Z': 30, '\ufeff': 31, '!': 32, '#': 33, '”': 34, 'N': 35, 'f': 36, ']': 37, 'á': 38, 'а': 39, '“': 40, 'v': 41, '%': 42, 'U': 43, '¥': 44, 'h': 45, 'b': 46, 'u': 47, 'Л': 48, 's': 49, 'O': 50, '|': 51, '°': 52, '¤': 53, 't': 54, 'G': 55, '£': 56, 'ő': 57, 'a': 58, '½': 59, '™': 60, '@': 61, 'о': 62, 'Q': 63, 'ö': 64, ';': 65, 'S': 66, '.': 67, '0': 68, '•': 69, '_': 70, '�': 71, '>': 72, 'o': 73, ':': 74, '»': 75, 'z': 76, ',': 77, '4': 78, 'n': 79, '{': 80, 'y': 81, '›': 82, '®': 83, '\xa0': 84, 'M': 85, 'A': 86, '7': 87, '-': 88, '/': 89, ' ': 90, 'C': 91, 'X': 92, '–': 93, '8': 94, 'j': 95, 'P': 96, 'V': 97, 'H': 98, 'E': 99, 'F': 100, 'W': 101, 'í': 102, '’': 103, 'L': 104, 'l': 105, 'è': 106, '<': 107, 'и': 108, '2': 109, 

ResourceExhaustedError:  OOM when allocating tensor with shape[138,2985,137] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[node gradient_tape/functional_9/dense_4/truediv/Neg (defined at <ipython-input-48-7438d94ff675>:9) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_14378]

Function call stack:
train_function


In [20]:
batch_size = 16
epochs = 1  

DataGenerator.__getitem__(1)

print(len(DataGenerator.encoder_input_data))
print(len(DataGenerator.encoder_input_data[0]))
print(len(DataGenerator.encoder_input_data[0][0]))

model.fit(
    [DataGenerator.encoder_input_data, DataGenerator.decoder_input_data],
    DataGenerator.decoder_target_data,
    batch_size=16,
    epochs=epochs,
    validation_split=0.2,
)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'tuple'

In [9]:
model.summary()

35
6119
394
Train on 28 samples, validate on 7 samples
28/28 [==============================] - 272s 10s/sample - loss: 6.0207 - accuracy: 0.4256 - val_loss: 5.7315 - val_accuracy: 0.99


In [1]:
model.save_weights("Model/test.weights.hdf5")
# keras.models.save_model(model, model_name)

NameError: name 'model' is not defined

In [10]:
def decode_sequence(DataGenerator: data_generator, input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, DataGenerator.tokens_count))
    target_seq[0, 0, DataGenerator.token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > DataGenerator.max_sequence_len:
            stop_condition = True

        target_seq = np.zeros((1, 1, DataGenerator.tokens_count))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [11]:
new_model = generate_model(num_encoder_tokens=DataGenerator.tokens_count, 
                           num_decoder_tokens=DataGenerator.tokens_count,
                           latent_dim=latent_dim)
new_model.compile(
    optimizer="rmsprop", 
    loss="categorical_crossentropy", 
    metrics=["accuracy"]
)

new_model.summary()

new_model.load_weights("Model/test.weights.hdf5")

In [12]:
encoder_inputs = new_model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = new_model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = new_model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_6")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_7")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = new_model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = new_model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in DataGenerator.token_index.items())
reverse_target_char_index = dict((i, char) for char, i in DataGenerator.token_index.items())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 61)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 61)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 325632      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  325632      input_4[0][0]                    
                                                                 lstm_2[0][1]               

In [16]:
for seq_index in range(5):
    print(seq_index)
    DataGenerator.__getitem__(0)
    input_seq = DataGenerator.encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(DataGenerator, input_seq)
    print("-")
    print("Input sentence:", DataGenerator.input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)
